In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from datetime import datetime, timedelta
from IPython.display import clear_output
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
start_data_season = {
    'M': 2003,
    'W': 2009
}
classifiers = {
    'M': RandomForestClassifier(max_depth=80, max_features='sqrt', min_samples_leaf=4,
                       n_estimators=1600, random_state=0),
    'W': RandomForestClassifier(max_depth=100, max_features=None, min_samples_leaf=4,
                       min_samples_split=5, n_estimators=2000, random_state=0)
}
columns = [
            'Season', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 
            'ASeedNum', 'ALastTournPct', 'ARegSznPct',
            'APts', 'APtsOpp', 'AFG', 'AFG3', 'AFT', 'AAst', 'ATO', 'AOR', 'ADR', 'AStl', 'ABlk', 'APF',
            'BSeedNum', 'BLastTournPct', 'BRegSznPct',
            'BPts', 'BPtsOpp', 'BFG', 'BFG3', 'BFT', 'BAst', 'BTO', 'BOR', 'BDR', 'BStl', 'BBlk', 'BPF',
            'Winner'
        ]
current_season = 2024

In [ ]:
def get_teams_info(gender):
    teams_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}Teams.csv'
    teams_df = pd.read_csv(teams_file)
    display(teams_df)
    return teams_df

m_teams_info_df = get_teams_info('M')
w_teams_info_df = get_teams_info('W')

In [ ]:
def get_seed_number(seed):
    if 'a' in seed or 'b' in seed:
        return 17
    return int(seed[1:])

def generate_numeric_cols(df, team):
    df[f'{team}FG'] = (df[f'{team}FGM'] * 100) / df[f'{team}FGA']
    df[f'{team}FG3'] = (df[f'{team}FGM3'] * 100) / df[f'{team}FGA3']
    df[f'{team}FT'] = (df[f'{team}FTM'] * 100) / df[f'{team}FTA']

def get_files_by_competition(gender):
    # Getting teams df
    teams_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}TeamConferences.csv'
    teams_df = pd.read_csv(teams_file)
    display(teams_df)
    
    # Getting regular season games df
    reg_szn_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}RegularSeasonDetailedResults.csv'
    reg_szn_df = pd.read_csv(reg_szn_file)
    generate_numeric_cols(reg_szn_df, 'W')
    generate_numeric_cols(reg_szn_df, 'L')
    display(reg_szn_df)
    
    # Getting tournament season games df
    tourn_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}NCAATourneyCompactResults.csv'
    tourn_df = pd.read_csv(tourn_file)
    display(tourn_df)
    
    # Getting seeding df
    seed_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}NCAATourneySeeds.csv'
    seed_df = pd.read_csv(seed_file)
    seed_df['SeedNum'] = seed_df.Seed.apply(lambda x: get_seed_number(x))
    display(seed_df)
    
    # Getting slots df
    slots_file = f'/kaggle/input/march-machine-learning-mania-2024/{gender}NCAATourneySlots.csv'
    slots_df = pd.read_csv(slots_file)
    display(slots_df)
    
    return teams_df, reg_szn_df, tourn_df, seed_df, slots_df

In [ ]:
def get_reg_szn_stats(season, team, reg_szn_df):
    w_games = reg_szn_df[reg_szn_df['WTeamID'] == team].rename(columns = {
        'WFG': 'FG', 'WFG3': 'FG3', 'WFT': 'FT', 'WAst': 'Ast', 'WTO': 'TO', 'WOR': 'OR', 'WDR': 'DR', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',
        'WScore': 'Score',
        
        'LFG': 'OppFG', 'LFG3': 'OppFG3', 'LFT': 'OppFT', 'LAst': 'OppAst', 'LTO': 'OppTO', 'LOR': 'OppOR', 'LDR': 'OppDR', 'LStl': 'OppStl', 'LBlk': 'OppBlk', 'LPF': 'OppPF',
        'LScore': 'OppScore',
        })
    w_games['Won'] = 1
    
    l_games = reg_szn_df[reg_szn_df['LTeamID'] == team].rename(columns = {
        'LFG': 'FG', 'LFG3': 'FG3', 'LFT': 'FT', 'LAst': 'Ast', 'LTO': 'TO', 'LOR': 'OR', 'LDR': 'DR', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF',
        'LScore': 'Score',
        
        'WFG': 'OppFG', 'WFG3': 'OppFG3', 'WFT': 'OppFT', 'WAst': 'OppAst', 'WTO': 'OppTO', 'WOR': 'OppOR', 'WDR': 'OppDR', 'WStl': 'OppStl', 'WBlk': 'OppBlk', 'WPF': 'OppPF',
        'WScore': 'OppScore',
        })
    l_games['Won'] = 0
    
    games = pd.concat([w_games, l_games], axis=0, ignore_index=True)
    
    reg_szn_pct = (len(w_games) * 100) / len(games)
    
    return [   
               reg_szn_pct, 
               games['Score'].mean(), games['OppScore'].mean(), games['FG'].mean(), games['FG3'].mean(), games['FT'].mean(), games['Ast'].mean(), games['TO'].mean(), 
               games['OR'].mean(), games['DR'].mean(), games['Stl'].mean(), games['Blk'].mean(), games['PF'].mean()
           ]

In [ ]:
def get_last_tourn_pct(season, team, tourn_df):
    wins = len(tourn_df[(tourn_df['Season'] == season-1) & (tourn_df['WTeamID'] == team)])
    total_games = len(tourn_df[(tourn_df['Season'] == season-1) & ((tourn_df['WTeamID'] == team) | (tourn_df['LTeamID'] == team))])
    
    if not total_games: return 0
    return (wins * 100) / total_games

In [ ]:
def set_team_name(team_id, teams_df):
    return teams_df[teams_df['TeamID'] == team_id].reset_index().loc[0, 'TeamName']

def get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df):
    a_reg_szn_stats = get_reg_szn_stats(season, team_a, reg_szn_df)
    b_reg_szn_stats = get_reg_szn_stats(season, team_b, reg_szn_df)

    try:
        a_seed = seed_df[(seed_df['Season'] == season) & (seed_df['TeamID'] == team_a)].reset_index().loc[0, 'SeedNum']
    except:
        a_seed = 24
    try:
        b_seed = seed_df[(seed_df['Season'] == season) & (seed_df['TeamID'] == team_b)].reset_index().loc[0, 'SeedNum']
    except:
        b_seed = 24

    a_last_tourn_pct = get_last_tourn_pct(season, team_a, tourn_df)
    b_last_tourn_pct = get_last_tourn_pct(season, team_b, tourn_df)

    stats_a = [a_seed, a_last_tourn_pct] + (a_reg_szn_stats)
    stats_b = [b_seed, b_last_tourn_pct] + (b_reg_szn_stats)
    
    teams_info_df = m_teams_info_df if gender == 'M' else w_teams_info_df
    team_a_name = set_team_name(team_a, teams_info_df)
    team_b_name = set_team_name(team_b, teams_info_df)
    
    return [season, team_a, team_b, team_a_name, team_b_name] + stats_a + stats_b

def build_dataset(gender, start_data_season, reg_szn_df, tourn_df, seed_df, end_season):
    data = []

    for season in range(start_data_season + 1, end_season + 1):
        tourney_games = tourn_df[tourn_df['Season'] == season].reset_index(drop=True)
        for idx, g in tourney_games.iterrows():

            clear_output(wait=True)
            print(f"{season}: {idx}/{len(tourney_games)}")

            team_a = min([g['WTeamID'], g['LTeamID']])
            team_b = max([g['WTeamID'], g['LTeamID']])

            if team_a == g['WTeamID']:
                winner = 'A'
                team_a_score = g['WScore']
                team_b_score = g['LScore']
            else:
                winner = 'B'
                team_a_score = g['LScore']
                team_b_score = g['WScore']

            print(f"{team_a} x {team_b}")

            game_stats = get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df)

            data.append(game_stats + [winner])

    data_df = pd.DataFrame(data, columns=columns)
    display(data_df)
    
    return data_df

In [ ]:
def train_model(x_train, y_train, gender):
    classifier = classifiers[gender]
    classifier.fit(x_train, y_train)
    return classifier

def predict_seasons(gender, start_data_season, data_df, detailed_results):

    acc_sum = 0
    seasons_count = 0

    for season in range(start_data_season + 2, current_season + 1):
        data_train = data_df[(data_df['Season'] < season)].reset_index(drop=True)
        data_test = data_df[(data_df['Season'] == season)].reset_index(drop=True)

        if not len(data_test):
            continue

        x_train = data_train.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
        y_train = data_train.loc[:, 'Winner']
            
        classifier = train_model(x_train, y_train, gender)

        x_test = data_test.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
        y_test = data_test.loc[:, 'Winner']

        predictions = classifier.predict(x_test)
        
        acc = accuracy_score(y_test, predictions)
        acc_sum += acc
        seasons_count += 1
        
        if detailed_results:
            print(f'\nResults for {gender} season {season}:')
            print('Accuracy predictions:', acc)

            cm = confusion_matrix(y_test, predictions)
            print('Confusion matrix:')
            cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classifier.classes_)
            cm_disp.plot()
            plt.show()

    print(f'\n\n{gender} Mean Accuracy: {acc_sum/seasons_count}')
    
    return classifier

In [ ]:
def get_current_predictions(gender, classifier, reg_szn_df, tourn_df, seed_df):
    data = []
    teams = seed_df[seed_df['Season'] == current_season].TeamID.unique()
    teams.sort()
    
    for idx, team_a in enumerate(teams):
        for idx_b in range(idx + 1, len(teams)):
            team_b = teams[idx_b]
            
            clear_output(wait=True)
            print(f"{idx}:{idx_b}/{len(teams)}")
            print(f"{team_a} x {team_b}")
            
            game_stats = get_game_stats(gender, current_season, team_a, team_b, reg_szn_df, seed_df, tourn_df)
            data.append(game_stats)
            
    data_df = pd.DataFrame(data, columns=columns[:-1])
    data_to_predict = data_df.drop(['TeamNameA', 'TeamNameB'], axis=1)
    
    probs = classifier.predict_proba(data_to_predict)
    pred = classifier.predict(data_to_predict)

    data_df['AProb'] = probs[:, 0]
    data_df['BProb'] = probs[:, 1]
    data_df['Pred'] = pred
    
    return data_df

In [ ]:
def pipeline(gender, detailed_results=False):
    teams_df, reg_szn_df, tourn_df, seed_df, slots_df = get_files_by_competition(gender)
    data_df = build_dataset(gender, start_data_season[gender], reg_szn_df, tourn_df, seed_df, current_season)
    classifier = predict_seasons(gender, start_data_season[gender], data_df, detailed_results)
    current_szn_df = get_current_predictions(gender, classifier, reg_szn_df, tourn_df, seed_df)
    display(current_szn_df)
    return current_szn_df, teams_df, slots_df

In [ ]:
m_current_szn_df, m_teams_df, m_slots = pipeline('M')

In [ ]:
w_current_szn_df, w_teams_df, w_slots = pipeline('W')

In [ ]:
def aggregate_probabilities_by_team(df):
    teams = pd.concat([df['TeamA'], df['TeamB']]).unique()
    team_probabilities = pd.DataFrame(index=teams, columns=teams)

    # Fill the win probability DataFrame with NaNs
    team_probabilities[:] = float('NaN')

    # Calculate win probabilities for each team against each other team
    for index, row in df.iterrows():
        team_probabilities.at[row['TeamA'], row['TeamB']] = row['AProb']
        team_probabilities.at[row['TeamB'], row['TeamA']] = row['BProb']

    # Create a new DataFrame to store the results
    result = pd.DataFrame(columns=['Team', 'Opponent', 'WinProb'])

    # Iterate over each team and opponent to fill the result DataFrame
    for team in teams:
        for opponent in teams:
            if team != opponent:
                win_prob = team_probabilities.at[team, opponent]
                result = pd.concat([result, pd.DataFrame([[team, opponent, win_prob]], columns=['Team', 'Opponent', 'WinProb'])], ignore_index=True)
    
    return result

probabilities_m = aggregate_probabilities_by_team(m_current_szn_df)
display(probabilities_m)

probabilities_w = aggregate_probabilities_by_team(w_current_szn_df)
display(probabilities_w)

In [ ]:
def get_win_probability_by_teams(probs_df, team, opponent):
    return probs_df.loc[(probs_df['Team'] == team) & (probs_df['Opponent'] == opponent), 'WinProb'].iloc[0]

In [ ]:
def build_slots(slots):
    slots = slots[slots['Season'] == current_season]
    slots = slots[slots['Slot'].str.contains('R')] 

    return slots

current_slots_m = build_slots(m_slots)
current_slots_w = build_slots(w_slots)

display(current_slots_m)
display(current_slots_w)

In [ ]:
def build_seeds_files():
    seeds_file = f'/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv'
    seeds_df = pd.read_csv(seeds_file)
    
    display(seeds_df)
    
    seeds_m = seeds_df[seeds_df['Tournament'] == 'M']
    seeds_w = seeds_df[seeds_df['Tournament'] == 'W']

    return seeds_m, seeds_w

seeds_m, seeds_w = build_seeds_files()

In [ ]:
def prepare_data(seeds):
    seed_dict = seeds.set_index('Seed')['TeamID'].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}

    return seed_dict, inverted_seed_dict


def simulate(round_slots, seeds, inverted_seeds, probabilites):
    winners = []
    slots = []

    for slot, strong, weak in zip(round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed):
        team_1, team_2 = seeds[strong], seeds[weak]

        team_1_prob = get_win_probability_by_teams(probabilites, team_1, team_2)
        winner = np.random.choice([team_1, team_2], p=[team_1_prob, 1 - team_1_prob])

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    return [inverted_seeds[w] for w in winners], slots


def run_simulation(seeds, round_slots, probabilites, brackets):
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict = prepare_data(seeds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probabilites)
        
        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({'Bracket': bracket, 'Slot': slots, 'Team': results})

    return result_df

In [ ]:
n_brackets = 50000

result_m = run_simulation(seeds_m, current_slots_m, probabilities_m, n_brackets)
result_m.insert(0, 'Tournament', 'M')

result_w = run_simulation(seeds_w, current_slots_w, probabilities_w, n_brackets)
result_w.insert(0, 'Tournament', 'W')

In [ ]:
submission = pd.concat([result_m, result_w])
submission.reset_index(inplace=True, drop=True)
submission.index.names = ['RowId']

display(submission)

submission.to_csv('submission.csv')